In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import tqdm.notebook as tqdm


# Project 1

### Task 1

Use the following probability matrix:
$$
\mathbf{P}=\left[\begin{array}{ccccc}
0.9915 & 0.005 & 0.0025 & 0 & 0.001 \\
0 & 0.986 & 0.005 & 0.004 & 0.005 \\
0 & 0 & 0.992 & 0.003 & 0.005 \\
0 & 0 & 0 & 0.991 & 0.009 \\
0 & 0 & 0 & 0 & 1
\end{array}\right]
$$

Assume that one time step equals one month. Simulate 1000 women, all
starting in state 1, until death. Summarize the lifetime distribution of the
women, after surgery, for example using a histogram. In what proportion of
women does the cancer eventually reappear, locally?

**Solution**

In [ ]:
P = np.array([[0.9915, 0.005, 0.0025, 0,     0.001],
              [0     , 0.986, 0.005,  0.004, 0.005],
              [0     ,0     , 0.992,  0.003, 0.005],
              [0     ,0     , 0,      0.991, 0.009],
              [0     ,0     , 0,      0,     1    ]])

# Probility of stating in the same state is along the diagonal
# Columns:
state_dict = {'no_tumor': 0,
              'local_recurrence': 1,
              'distant_metastatis': 2,
              'local_and_distant': 3,
              'death': 4}

In [ ]:
iterations = 1000
n_woman = 1000
states = np.zeros(1000, dtype=int)

lifetimes = np.zeros(1000, dtype=int) -1

state_dist = np.zeros((5, 200*12)) # 200 years

local_recurrence_counter = 0
t = 0
for i in range(iterations):
     while np.sum(states) < n_woman * state_dict['death']:
          for i, state in enumerate(states):
               
               state_dist[state, t] += 1

               if state == state_dict['death']:
                    continue

               new_state = np.random.choice(5, p=P[state])
               
               if new_state == state_dict['death']:
                    assert lifetimes[i] == -1, "someone died twice"
                    lifetimes[i] = t

               # if state is no tumor or distant metastatis, it can go to local recurrence
               if state == state_dict['no_tumor'] or state == state_dict['distant_metastatis']:
                    if new_state == state_dict['local_recurrence'] or new_state == state_dict['local_and_distant']:
                         local_recurrence_counter += 1
                    
               states[i] = new_state
          t += 1

assert np.all(lifetimes != -1), "someone did not die"

state_dist /= iterations

print(f'Local recurrence rate: {local_recurrence_counter / n_woman * 100:.2f}%')

fig, axs = plt.subplots(1, 5, figsize=(20, 5))

for i, ax in enumerate(axs):
     bins = np.argmax(np.argwhere(state_dist[i] > 0))+1
     ax.plot(np.arange(len(state_dist[i,:bins])),state_dist[i,:bins])
     ax.set_title(list(state_dict.keys())[i])
     ax.set_xlabel('Months')
plt.show()


### Task 2

Let $\mathbf{p}_t$ denote the probability distribution over the states at time $t$. This
distribution can be found from the following formula:

$$
\mathbf{p}_t=\mathbf{p}_0\left(\mathbf{P}^t\right)
$$

In your simulations, what is the distribution over the states at $t = 120$?
Does this correspond to what we expect? Answer the question using an
appropriate statistical test.

In [ ]:
p_120_sim = state_dist[:,120]
p_120_true = (np.array([[1., 0. ,0., 0., 0.]])@np.linalg.matrix_power(P, 120))[0]

# chi2 test
chi2, p = stats.chisquare(p_120_sim, p_120_true)

print(f'Chi2: {chi2}, p-value: {p}')

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].bar(np.arange(5),p_120_sim)
axs[0].set_title('Simulation')
axs[1].bar(np.arange(5),p_120_true)
axs[1].set_title('True')
plt.show()


### Task 3

The above approach only validates the simulation at a certain time point. A
better approach is to ensure the emperical lifetime distribution matches the
theoretical.
It can be shown that the lifetime, $T$, follows a so-called discrete phase-type
distribution. This distribution has probability mass function

$$P(T=t)=\boldsymbol{\pi}\left(\mathbf{P}_{\mathrm{s}}\right)^t \mathbf{p}_{\mathbf{s}}$$

and mean

$$E(T)=\boldsymbol{\pi}\left(\mathbf{I}-\mathbf{P}_s\right)^{-1} \mathbf{1}$$

Where $\boldsymbol{\pi}$ is the distribution over states $1, 2, 3, 4$ at $t = 0$. $\mathbf P_s$ is a $4\times 4$
sub-matrix of $\mathbf P$ formed by removing the last row and column. $\mathbf p_s$ is column
vector indicating the probability of dying from states $1, 2, 3, 4$. $\mathbf 1$ is a vector
of ones of appropriate dimension.

Does your simulated lifetimes follow this distribution?

**Solution:**

In [ ]:
pi = np.zeros((4, 1))
pi[0] = 1

Ps = P[:4,:4]
ps = P[:4,-1][..., None]

phase_type_dist = lambda t : (pi.T @ np.linalg.matrix_power(Ps, t) @ ps).item()
phase_type_mean = pi*np.linalg.inv(np.identity(4) - Ps)@np.ones((4,1))

bins = np.argmax(np.argwhere(state_dist[-1,1:] == 0))+1
death_dist_pred = state_dist[-1,:bins]
death_pred_true = np.array([phase_type_dist(t) for t in range(bins)])

cumulative_death_dist_true = np.cumsum(death_pred_true)


# chi2 test
max_length = 1000
f_obs = np.zeros(max_length, dtype=int)
length, count = np.unique(lifetimes, return_counts=True)

less_than_max_mask = length < max_length
f_obs[length[less_than_max_mask]] = count[less_than_max_mask]
f_obs[-1] = np.sum(count[~less_than_max_mask])

f_exp = np.array([phase_type_dist(t) for t in range(max_length)])
f_exp[-1] = 1 - np.sum(f_exp[:-1])
f_exp *= len(lifetimes)



# plot the results
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.plot(np.arange(bins), cumulative_death_dist_true, label='Theoretical')
ax.plot(np.arange(bins), death_dist_pred, label='Simulation')
ax.set_title(f'Cumulative death distribution, chi2: p-value={stats.chisquare(f_obs, f_exp).pvalue:.2f}')
ax.set_xlabel('Months')
ax.legend()
plt.show()

In [ ]:
stats.chisquare(f_obs, f_exp)

### Task 4

Estimate the expected lifetime, after surgery, of a woman who survives the
first 12 months following surgery, but whose breast cancer has also reappeared within the first 12 months, either locally or distant.

*Hint:* Use rejection sampling. Simulate a number of women, and discard
all simulations that do not meet the requirements. Do this until you have
reached 1000 acceptable simulations.

**Solution:**

In [ ]:
def simulate_lifetime(P, state_dict):
    state = state_dict['no_tumor']
    lifetime = 0
    recurrence = False
    while state != state_dict['death']:
        new_state = np.random.choice(5, p=P[state])
        if lifetime < 12:
            if new_state in [state_dict['local_recurrence'], state_dict['distant_metastatis'], state_dict['local_and_distant']]:
                recurrence = True
        state = new_state
        lifetime += 1
    return lifetime, recurrence

# Use rejection sampling
acceptable_lifetimes = []
iterations = 0
required_samples = 1000

pbar = tqdm.tqdm(total=required_samples, leave=False)
while len(acceptable_lifetimes) < required_samples:
    lifetime, recurrence = simulate_lifetime(P, state_dict)
    if lifetime > 12 and recurrence:
        pbar.update(1)
        acceptable_lifetimes.append(lifetime)
    iterations += 1
pbar.close()

expected_lifetime = np.mean(acceptable_lifetimes)
print(f'Expected lifetime after surgery for acceptable cases: {expected_lifetime} months')

# plot the distribution of lifetimes for acceptable cases
plt.hist(acceptable_lifetimes, bins=30, edgecolor='black')
plt.title('Distribution of Lifetimes for Acceptable Cases')
plt.xlabel('Lifetime (months)')
plt.ylabel('Frequency')
plt.show()

### Task 5
What fraction of women die within the first 350 months? Answer this by
simulating 200 women, and record the fraction. Do this 100 times. Use control variates to reduce the variance.
How large a reduction in variance do you see, using control variates, as opposed to the crude Monte Carlo estimator?

*Hint:* Use the mean lifetime after surgery of the 200 simulations as the
control variate

**Solution:**

In [ ]:
def simulate_lifetime(P, state_dict):
    state = state_dict['no_tumor']
    lifetime = 0
    while state != state_dict['death']:
        new_state = np.random.choice(5, p=P[state])
        state = new_state
        lifetime += 1
    return lifetime


def simulation_200_women(P, state_dict):
    lifetimes = []
    for _ in range(200):
        lifetime = simulate_lifetime(P, state_dict)
        lifetimes.append(lifetime)
    lifetimes = np.array(lifetimes)
    fraction_died_within_350 = np.mean(lifetimes <= 350)
    mean_lifetime = np.mean(lifetimes)
    return fraction_died_within_350, mean_lifetime


# Perform 100 simulations
fractions = []
control_means = []
for _ in tqdm.trange(100, leave=False):
    fraction, mean_lifetime = simulation_200_women(P, state_dict)
    fractions.append(fraction)
    control_means.append(mean_lifetime)

fractions = np.array(fractions)
control_means = np.array(control_means)


# Crude Monte Carlo estimate
mean_fraction = np.mean(fractions)
variance_crude = np.var(fractions)

# Control variates method
expected_mean_lifetime = np.mean([simulate_lifetime(P, state_dict) for _ in range(1000)])
c = np.cov(fractions, control_means)[0, 1] / np.var(control_means)
adjusted_fractions = fractions - c * (control_means - expected_mean_lifetime)
mean_adjusted_fraction = np.mean(adjusted_fractions)
variance_adjusted = np.var(adjusted_fractions)

# Results
print(f'Crude Monte Carlo estimate of fraction: {mean_fraction}')
print(f'Variance of crude estimate: {variance_crude}')
print(f'Control variates estimate of fraction: {mean_adjusted_fraction}')
print(f'Variance of control variates estimate: {variance_adjusted}')
print(f'Reduction in variance: {variance_crude - variance_adjusted}')
print(f'Percentage reduction in variance: {100 * (variance_crude - variance_adjusted) / variance_crude}%')

# Optionally, plot the results
plt.hist(fractions, bins=30, alpha=0.5, label='Crude Monte Carlo')
plt.hist(adjusted_fractions, bins=30, alpha=0.5, label='Control Variates')
plt.title('Distribution of Fraction of Women Dying within 350 Months')
plt.xlabel('Fraction')
plt.ylabel('Frequency')
plt.legend()
plt.show()

### Task 6

For the report, consider the following questions: What assumptions underlie
the discrete time Markov chain model? Are those assumptions realistic? How
may we relax some of these assumptions, possibly at the cost of increased
model complexity?

**Solution:**

## Part 2: A continuous-time model

*see project text*

### Task 7

As before, we choose one time-unit equal to one month. Use the following
transition-rate matrix:

$$
\mathbf{Q}=\left[\begin{array}{ccccc}
-0.0085 & 0.005 & 0.0025 & 0 & 0.001 \\
0 & -0.014 & 0.005 & 0.004 & 0.005 \\
0 & 0 & -0.008 & 0.003 & 0.005 \\
0 & 0 & 0 & -0.009 & 0.009 \\
0 & 0 & 0 & 0 & 0
\end{array}\right]
$$

Simulate 1000 women, all starting in state 1, until death. Summarize the
lifetime distribution after surgery, for example in a histogram. Report the
mean, along with a confidence interval, and the standard deviation, also
with a confidence interval. In what proportion of women has the cancer
reappeared distantly after 30.5 months?

**Solution:**